<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-06 13:15:10
-------------------
qualified stocks: 92
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.53 L
Current:  1.35 C
-------------------
Today PnL: -91.50 K (-0.67%)
Current PnL: -19.81 L (-13.47%)
CY Booked + Current PnL: -11.54 L (-7.85%)
-------------------
Total profit:  2.15 L
Total loss:  -21.96 L
-------------------
Total Booked + Current PnL: 15.44 L (12.86%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.12%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 78.08 L (57.81%)
Deployed:  1.20 C
Current:  1.35 C
CAGR/XIRR %: 7.69%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,761.55,-19.90,H-LC,5.31,110384.0,6072.0,11469.0,0.41,5.82,10.39,16.81,10.0,0.53,0.84,22.80,X5K,ATH,METALS
67,SBIN,760.30,863.00,-11.51,M-LC,3.98,213855.0,12376.0,14842.0,0.80,6.14,6.94,13.51,62.0,0.83,1.63,19.51,XY25,NTT,BANKS
82,TTKPRESTIG,769.29,770.00,-9.57,H-SC,11.65,82700.0,-18077.0,18169.0,0.43,-17.94,21.97,0.09,145.0,-0.99,0.63,8.65,OX40N,NTT,DURABLES
71,SIS,477.00,477.00,2237.82,H-SC,7.00,62900.0,-18190.0,18191.0,-1.11,-22.43,28.92,0.00,157.0,-1.00,0.48,27.48,OX40N,NTT,MISC
27,DIXON,14367.00,18940.15,-27.76,H-MC,6.63,132984.0,18048.0,18538.0,-1.34,15.70,13.94,31.83,85.0,0.97,1.01,49.66,X40N,ATH,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,NATIONALUM,189.63,247.44,-50.15,H-MC,2.87,99847.0,-657.0,31292.0,0.60,-0.65,31.34,30.49,81.0,-0.02,0.76,31.54,MH,ATH,METALS
91,WIPRO,243.46,420.00,-12.39,M-LC,5.88,150096.0,-849.0,110306.0,-1.55,-0.56,73.49,72.51,55.0,-0.01,1.14,5.43,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.50,H-LC,1.82,203938.0,-562.0,41970.0,-0.75,-0.27,20.58,20.25,9.0,-0.01,1.55,8.43,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-14.41,H-LC,2.65,170129.0,-31257.0,118869.0,2.67,-15.52,69.87,43.51,19.0,-0.26,1.29,18.43,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-5.08,H-LC,3.21,182862.0,-17448.0,77424.0,-1.01,-8.71,42.34,29.94,24.0,-0.23,1.39,26.33,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-10.42,H-LC,3.74,186074.0,-14432.0,63209.0,-0.67,-7.20,33.97,24.33,30.0,-0.23,1.41,7.30,X40N,BTT,FINANCE
57,NESTLEIND,2268.60,2755.0,-17.42,H-LC,4.19,261741.0,-3685.0,60593.0,-1.33,-1.39,23.15,21.44,26.0,-0.06,1.99,5.16,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,2961.56,4250.00,-14.41,H-LC,2.65,170129.0,-31257.0,118869.0,2.67,-15.52,69.87,43.51,19.0,-0.26,1.29,18.43,X40,BTT,PAINTS
84,UNIONBANK,123.87,163.00,-15.25,M-LC,15.61,148799.0,7959.0,36530.0,2.26,5.65,24.55,31.59,72.0,0.22,1.13,33.64,XY24,NTT,BANKS
2,ACC,2579.05,3906.00,-38.07,M-MC,4.88,133524.0,-54747.0,151617.0,1.51,-29.08,113.55,51.45,178.0,-0.36,1.02,2.28,XY24,BTT,CEMENT
42,INDIAMART,2327.09,4850.92,-48.64,H-SC,13.40,133555.0,10219.0,123538.0,1.33,8.29,92.50,108.45,118.0,0.08,1.02,32.49,AR,ATH,MISC
12,BANKINDIA,116.91,190.00,-34.97,H-MC,9.53,172348.0,-7460.0,119868.0,1.21,-4.15,69.55,62.52,92.0,-0.06,1.31,28.52,XR,NTT,BANKS


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BRITANNIA,5502.66,6362.67,8.51,H-LC,9.31,194796.0,-3300.0,34265.0,-3.98,-1.67,17.59,15.63,31.0,-0.10,1.48,19.82,XY25,ATH,FMCG
60,RAJESHEXPO,517.65,518.00,1665.15,L-SC,3.42,48166.0,-89011.0,89102.0,-3.89,-64.89,184.99,0.07,266.0,-1.00,0.37,19.79,OX40N,NTT,JEWELLERY
75,SYMPHONY,1306.42,1306.00,-39.43,M-SC,9.43,129232.0,-41909.0,41858.0,-3.30,-24.49,32.39,-0.03,197.0,-1.00,0.98,0.00,OX40N,NTT,DURABLES
74,SURYODAY,189.78,240.00,40.35,H-SC,5.05,99303.0,-50623.0,90296.0,-3.11,-33.77,90.93,26.46,142.0,-0.56,0.75,27.21,XR,NTT,BANKS
76,TANLA,917.30,1963.11,-30.09,H-SC,7.64,171020.0,-68395.0,341356.0,-2.51,-28.57,199.60,114.01,133.0,-0.20,1.30,56.85,AR,ATH,IT


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.0,127.68,H-SC,4.68,149259.0,-25300.0,25329.0,-0.11,-14.49,16.97,0.02,135.0,-1.00,1.13,29.43,OX40N,NTT,PAINTS
82,TTKPRESTIG,769.29,770.0,-9.57,H-SC,11.65,82700.0,-18077.0,18169.0,0.43,-17.94,21.97,0.09,145.0,-0.99,0.63,8.65,OX40N,NTT,DURABLES
83,UJJIVANSFB,52.77,53.0,34.19,H-SC,20.65,116478.0,-26001.0,26627.0,-1.93,-18.25,22.86,0.44,163.0,-0.98,0.89,37.48,OX40N,NTT,BANKS
77,TATAELXSI,7332.28,7332.0,-21.51,H-MC,2.04,82355.0,-20297.0,20292.0,-1.33,-19.77,24.64,-0.00,101.0,-1.00,0.63,25.19,OX40N,NTT,IT
49,KANSAINER,299.63,340.0,-69.48,H-SC,1.66,212580.0,-57087.0,93429.0,-2.47,-21.17,43.95,13.47,136.0,-0.61,1.62,7.52,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-27.06,H-LC,14.86,237206.0,-58728.0,152642.0,-0.74,-19.85,64.35,31.73,1.0,-0.38,1.80,1.27,X40,BTT,IT
88,VBL,492.64,671.64,-11.22,H-LC,11.34,269619.0,4579.0,91724.0,-1.37,1.73,34.02,36.33,3.0,0.05,2.05,15.26,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-19.60,H-LC,8.66,258369.0,-3232.0,148846.0,-1.16,-1.24,57.61,55.67,4.0,-0.02,1.96,4.79,X40,BTT,IT
1,ABB,5551.76,7934.00,-44.34,H-LC,8.74,198666.0,-17853.0,110756.0,0.20,-8.25,55.75,42.91,5.0,-0.16,1.51,3.84,AR,NTT,ELECTRICAL
34,HCLTECH,1562.67,1943.91,0.97,H-LC,9.08,178803.0,-11843.0,58361.0,-1.15,-6.21,32.64,24.40,8.0,-0.20,1.36,8.73,X40,ATH,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,19.85,H-SC,13.44,153677.0,27477.0,18702.0,-0.15,21.77,12.17,36.59,156.0,1.47,1.17,37.02,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,29.10,H-SC,13.48,132023.0,6726.0,52347.0,-1.61,5.37,39.65,47.15,144.0,0.13,1.00,65.17,MH,ATH,POWER
55,MEDANTA,1087.93,1486.00,19.85,H-SC,13.44,153677.0,27477.0,18702.0,-0.15,21.77,12.17,36.59,156.0,1.47,1.17,37.02,XY24,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-2.38,H-MC,4.93,157227.0,6314.0,50989.0,-1.20,4.18,32.43,37.97,90.0,0.12,1.20,30.55,AR,ATH,AUTO
19,CAMS,3643.00,5226.82,-4.58,H-SC,1.76,107260.0,5256.0,39096.0,0.90,5.15,36.45,43.48,120.0,0.13,0.82,22.80,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2326.38,-6.81,H-SC,8.68,174800.0,10130.0,57841.0,-0.15,6.15,33.09,41.28,119.0,0.18,1.33,14.63,X40N,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,29.10,H-SC,13.48,132023.0,6726.0,52347.0,-1.61,5.37,39.65,47.15,144.0,0.13,1.00,65.17,MH,ATH,POWER
35,HEROMOTOCO,4311.81,5949.07,-2.38,H-MC,4.93,157227.0,6314.0,50989.0,-1.20,4.18,32.43,37.97,90.0,0.12,1.20,30.55,AR,ATH,AUTO
90,WHIRLPOOL,1219.98,2270.00,-41.46,M-SC,5.72,98558.0,7060.0,71691.0,-0.23,7.72,72.74,86.07,211.0,0.10,0.75,42.57,XR,NTT,DURABLES
42,INDIAMART,2327.09,4850.92,-48.64,H-SC,13.40,133555.0,10219.0,123538.0,1.33,8.29,92.50,108.45,118.0,0.08,1.02,32.49,AR,ATH,MISC
91,WIPRO,243.46,420.00,-12.39,M-LC,5.88,150096.0,-849.0,110306.0,-1.55,-0.56,73.49,72.51,55.0,-0.01,1.14,5.43,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-55.73,L-SC,21.62,66586.0,-46963.0,87015.0,-1.56,-41.36,130.68,35.27,267.0,-0.54,0.51,62.97,XR,NTT,HOTELS
54,MASFIN,326.60,399.50,-19.15,H-SC,16.33,91695.0,-6285.0,28160.0,1.19,-6.41,30.71,22.32,151.0,-0.22,0.70,32.99,XR,ATH,FINANCE
3,ALKYLAMINE,2347.98,4546.37,0.20,H-SC,18.82,93254.0,-7709.0,102244.0,-2.04,-7.64,109.64,93.63,149.0,-0.08,0.71,38.90,SR,ATH,CHEMICALS
87,VALIANTORG,512.64,838.00,-322.71,H-SC,19.80,94796.0,-38490.0,123083.0,0.00,-28.88,129.84,63.47,147.0,-0.31,0.72,58.90,XR,NTT,CHEMICALS
36,HINDALCO,651.95,761.55,-19.90,H-LC,5.31,110384.0,6072.0,11469.0,0.41,5.82,10.39,16.81,10.0,0.53,0.84,22.80,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LALPATHLAB,2838.53,3545.00,-23.54,H-SC,7.01,217688.0,24668.0,23380.0,0.40,12.78,10.74,24.89,128.0,1.06,1.66,39.11,X40N,NTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,19.85,H-SC,13.44,153677.0,27477.0,18702.0,-0.15,21.77,12.17,36.59,156.0,1.47,1.17,37.02,XY24,NTT,HEALTHCARE
69,SHALBY,261.39,327.00,936.68,M-SC,8.42,140963.0,-40180.0,85649.0,-1.65,-22.18,60.76,25.10,233.0,-0.47,1.07,12.97,XY24,NTT,HEALTHCARE
88,VBL,492.64,671.64,-11.22,H-LC,11.34,269619.0,4579.0,91724.0,-1.37,1.73,34.02,36.33,3.0,0.05,2.05,15.26,X40N,ATH,FMCG
27,DIXON,14367.00,18940.15,-27.76,H-MC,6.63,132984.0,18048.0,18538.0,-1.34,15.70,13.94,31.83,85.0,0.97,1.01,49.66,X40N,ATH,IT


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.70
1,25,41.50
2,50,71.39


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.66
LC    37.16
MC    22.19
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.96
X40N     15.19
X40      13.26
XY25     11.69
XR        8.99
AR        7.71
OX40N     7.68
X5K       2.23
MH        1.76
X200      1.39
SR        1.15
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    29.00
H-SC    27.37
H-MC    19.21
M-SC    11.87
M-LC     7.10
M-MC     2.63
L-SC     1.42
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.21,-3.04,36.16
IT,12.01,-17.98,75.28
FINANCE,8.36,-18.70,61.77
BANKS,7.52,-15.30,62.35
PAINTS,5.79,-14.56,37.14
MISC,5.11,-21.62,92.77
HEALTHCARE,5.10,2.12,22.60
ELECTRICAL,4.71,-13.46,62.60
AUTO,4.41,-16.96,67.84


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2829329.0,23
XR,1110631.0,13
AR,1052334.0,9
X40,826609.0,10
X40N,609966.0,12
XY25,510919.0,8
OX40N,368553.0,10
SR,192359.0,2
X5K,146044.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2724870.0,30
M-SC,1408674.0,15
H-LC,1338058.0,20
H-MC,1180294.0,15
M-LC,445217.0,5
M-MC,332231.0,2
L-SC,275014.0,3
L-MC,56451.0,1
L-LC,46998.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      948419.0      7
M-SC       XY24      860866.0      7
H-SC       AR        610680.0      4
H-LC       X40       548218.0      6
H-SC       XR        517523.0      6
H-MC       XY24      461229.0      4
M-MC       XY24      332231.0      2
H-LC       X40N      311276.0      6
           AR        232498.0      2
H-SC       X40N      215480.0      4
H-MC       XY25      211238.0      2
           X40       202176.0      3
H-SC       SR        192359.0      2
           OX40N     188062.0      6
L-SC       XR        185912.0      2
M-LC       XY24      185494.0      2
M-SC       AR        158167.0      2
M-LC       X5K       134575.0      1
M-SC       XY25      121758.0      1
           XR        120571.0      2
H-MC       XR        119868.0      1
H-LC       XY25      116083.0      3
M-LC       XR        110306.0      1
L-SC       OX40N      89102.0      1
H-MC       X40N       83210.0      2
H-LC       X200       77424.0      1
M-SC       X40        76215.0      1
           OX40N      71097.0      2
L-MC       XR         56451.0      1
H-SC       MH         52347.0      1
H-MC       AR         50989.0      1
L-LC       XY25       46998.0      1
H-LC       XY24       41090.0      1
H-MC       MH         31292.0      1
           OX40N      20292.0      1
M-LC       XY25       14842.0      1
H-LC       X5K        11469.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
